In [1]:
from starlette.applications import Starlette
from starlette.responses import HTMLResponse, JSONResponse
from starlette.staticfiles import StaticFiles
import sys
sys.path.append('../recommender/')
from starlette.middleware.cors import CORSMiddleware
import uvicorn, aiohttp, asyncio
from io import BytesIO
from pathlib import Path
from cloth_detector import *
from style_recommender import *
import email_agent as ea

In [ ]:
app = Starlette()
app.add_middleware(CORSMiddleware, allow_origins=['*'], allow_headers=['X-Requested-With', 'Content-Type'])
app.mount('/static', StaticFiles(directory='../static'))
app.mount('/static', StaticFiles(directory='../static/image_data'))
app.mount('/static', StaticFiles(directory='../static/tmp'))


@app.route('/')
def index(request):
    html = Path("../view/index.html")
    return HTMLResponse(html.open().read())

@app.route('/email', methods=['POST'])
async def send_email(request):
    json = await request.json()
    await ea.send_email(json["name"],json["email"],json["comments"])
    print(json)
    return JSONResponse({})

@app.route('/analyze', methods=['POST'])
async def detect_items(request):
    json = await request.json()
    result = get_recommendations(json)
    print(result)
    return JSONResponse({"results":result})

@app.route('/detect', methods=['POST'])
async def analyze(request):
    data = await request.form()
    img_bytes = await (data['file'].read())
    response = get_detected_items(Image.open(BytesIO(img_bytes)))
    print(response)
    return JSONResponse(response)

if __name__ == '__main__':
    uvicorn.run(app=app, host='0.0.0.0', port=1997)

INFO: Started server process [2685]
INFO: Waiting for application startup.
INFO: Uvicorn running on http://0.0.0.0:1997 (Press CTRL+C to quit)
INFO: ('90.208.244.61', 56264) - "GET / HTTP/1.1" 200
INFO: ('90.208.244.61', 56264) - "GET /static/example.jpg HTTP/1.1" 200
INFO: ('90.208.244.61', 56266) - "GET /static/image-picker.css HTTP/1.1" 304
INFO: ('90.208.244.61', 56268) - "GET /static/example-cropped1.jpg HTTP/1.1" 200
INFO: ('90.208.244.61', 56272) - "GET /static/client.js HTTP/1.1" 304
INFO: ('90.208.244.61', 56274) - "GET /static/image-picker.min.js HTTP/1.1" 304
INFO: ('90.208.244.61', 56266) - "GET /static/scroll-reveal-conf.js HTTP/1.1" 304
INFO: ('90.208.244.61', 56276) - "GET /static/style.css HTTP/1.1" 304
INFO: ('90.208.244.61', 56272) - "GET /static/example-cropped4.jpg HTTP/1.1" 304
INFO: ('90.208.244.61', 56274) - "GET /static/example-cropped3.jpg HTTP/1.1" 304
INFO: ('90.208.244.61', 56266) - "GET /static/example-cropped2.jpg HTTP/1.1" 304
INFO: ('31.13.115.13', 64536